## Лабораторная работа № 1 
## Выполнение разведочного анализа больших данных с использованием фреймворка Apache Spark

### Часть 2

В данной части работы рассмотрены:
* разведочный анализ данных;
* работа с Dataframe API фреймворка `Apache Spark`.

Подключаем необходимые библиотеки.

In [139]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    col, lit, sum, mean, when,
    explode, count, desc, floor,
    corr, array_contains, lit, first
)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [9]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 1 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [10]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Укажем базу данных, которая была создана в первой части лабораторной работы.

In [140]:
database_name = "Efremenkov_database"

Установим созданную базу данных как текущую.

In [141]:
spark.catalog.setCurrentDatabase(database_name)

Прочитаем сохранённую в предыдущей части работы таблицу и загрузим её в `Spark Dataframe`.

In [142]:
df = spark.table("sobd_lab1_table")

Выведем прочитанную таблицу на экран.

In [ ]:
df.show()

Посмотрим на схему данных.

In [ ]:
df.printSchema()

Вычислим количество строк в датафрейме.

In [ ]:
df.count()

#### Анализ столбца `_c0`
Поскольку первичным ключём является столбец `LMK_KEY`, то номер строки нам не нужен

In [ ]:
df = df.drop("_c0")
df.show()

#### Анализ столбца `LMK_KEY`

Отсортируем датафрейм по столбцу `LMK_KEY`

In [ ]:
df.orderBy("LMK_KEY", ascending=True).show()

Видно, что некоторые значения ключа довольно странные (представляют собой больше описание, чем ключ). Удалим из датафрейма строки с такими ключами, оставив только подходящие по шаблону.

In [ ]:
# Регулярное выражение для VIN длиной 17 символов, состоящих из цифр и букв
key_pattern = r"^[a-f0-9]+$"

# Фильтрация DataFrame
df = df.filter(col("LMK_KEY").rlike(key_pattern))
df.show()

In [ ]:
df.count()

Проверим наличие дубликатов в датафрейме.

In [ ]:
(
    df
    .groupBy("LMK_KEY")
    .count()
    .where("count > 1")
    .orderBy("count", ascending=False)
    .show()
)

Дубликатов нет.

#### Анализ столбца `ADDRESS`

Посмотрим внимательно на значения в столбце. Видно, что в данном столбце расположен **категориальный признак**.

Введем функцию, определяющую количество NULL-значений в столбце.

In [150]:
def count_nulls(data: DataFrame,
                column_name: str) -> None:
    """
    Подсчет количества null и not null значений в указанном столбце.

    Args:
        data (DataFrame): DataFrame, содержащий данные.
        column_name (str): Имя столбца для подсчета null и not null значений.
    
    Returns:
        None
    """
    # Подсчет количества null значений в указанном столбце
    null_counts = data.select(
        sum(col(column_name).isNull().cast("int"))
    ).collect()[0][0]

    # Подсчет количества not null значений в указанном столбце
    not_null_counts = data.select(
        sum(col(column_name).isNotNull().cast("int"))
    ).collect()[0][0]

    # Вывод результатов
    print(f"Число строк с NULL: {null_counts} "
          f"({100 * null_counts / (null_counts + not_null_counts):.2f}%)")

In [ ]:
count_nulls(data=df, column_name="ADDRESS")

Заменим пропущенные значения на Unknown

In [ ]:
df = df.fillna({"ADDRESS": "Unknown"})
count_nulls(data=df, column_name="ADDRESS")

#### Анализ столбца `ADDRESS2`

In [ ]:
count_nulls(data=df, column_name="ADDRESS2")

Как видим, ситуация аналогичная. Удаляем столбец.

In [ ]:
df = df.drop("ADDRESS2")
df.show()

#### Анализ столбца `ADDRESS3`

In [ ]:
count_nulls(data=df, column_name="ADDRESS3")

Видно, что более 90% данных в столбце пропущены. Можно, конечно, попытаться обработать то, что есть, но в целях упрощения анализа просто удалим столбец.

In [ ]:
df = df.drop("ADDRESS3")
df.show()

#### Анализ столбца `CURRENT_ENERGY_EFFICIENCY` - колличественный признак


In [ ]:
count_nulls(data=df, column_name="CURRENT_ENERGY_EFFICIENCY")

Пустых значений не обнаружено

#### Анализ столбца `PROPERTY_TYPE`

Посмотрим внимательно на значения в столбце. Видно, что в данном столбце расположен **категориальный признак**.

Введем функцию, определяющую количество NULL-значений в столбце.

In [ ]:
count_nulls(data=df, column_name="PROPERTY_TYPE")

Видно, что столбцы `PROPERTY_TYPE` не содержит пропущенных значений. Если бы признак их содержал, то логично заменить пропущенные значения на категорию `Unknown`.

In [ ]:
df = df.fillna({"PROPERTY_TYPE": "Unknown"})
count_nulls(data=df, column_name="PROPERTY_TYPE")

Создадим функцию расчета и визуализации распределения категориальных признаков.

In [161]:
def plot_cat_distribution(data: DataFrame,
                          column_name: str,
                          top_n: int = 20) -> None:
    """
    Построение распределения категориального признака.

    Args:
        data (DataFrame): DataFrame, содержащий данные.
        column_name (str): Имя столбца для группировки.
        top_n (int): Количество топ-значений для отображения.
    
    Returns:
        None
    """
    # Группировка данных по столбцу и подсчет количества
    categories = (
        data
        .groupBy(column_name)
        .count()
        .orderBy("count", ascending=False)
    )
    
    print(f"Количество категорий признака {column_name}: {categories.count()}")

    categories = (
        categories
        .limit(top_n)
        .toPandas()
    )
    
    # Визуализация с использованием Seaborn
    plt.figure(figsize=(10, 6))
    sns.barplot(x=column_name, y="count", data=categories)
    plt.title(f"Barplot of \"{column_name}\" counts")
    plt.xlabel(column_name)
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
plot_cat_distribution(data=df, column_name="PROPERTY_TYPE")

Видно, что недвижимость в представленном датасете имеются 5 определенных типов мест проживания.

#### Анализ столбца `INSPECTION_DATE`
Признак **дата**. Уберем пустые значения при наличии

In [ ]:
count_nulls(data=df, column_name="INSPECTION_DATE")
df.show()

#### Анализ столбца `HEATING_COST_CURRENT`

Данный столбец, согласно описанию и значениям, которые он принимает, можно отнести к **количественным**.

Выполним аналогичные шаги.

In [ ]:
count_nulls(data=df, column_name="HEATING_COST_CURRENT")

Пропущено много значений, но меньше половины. Стоит попытаться сохранить признак и обработать его.

In [ ]:
plot_boxplots(data=df, columns=["HEATING_COST_CURRENT"])

Можно наблюдать сильные выбросы. Удалим строки, их содержащие, и убедимся, что потеряна небольшая часть данных.

In [ ]:
df.filter(col("HEATING_COST_CURRENT") > 8000).count()

Заменим пропуски средним значением признака.

In [ ]:
df = df.filter(col("HEATING_COST_CURRENT") < 8000)
mean_cost = df.select(mean(col("HEATING_COST_CURRENT"))).collect()[0][0]
mean_cost

In [168]:
df = df.fillna({"HEATING_COST_CURRENT": mean_cost})

Вырежем также строки со значениями <= 0

In [ ]:
df.filter(col("HEATING_COST_CURRENT") <= 0).count()

In [170]:
df = df.filter(col("HEATING_COST_CURRENT") > 0)

In [ ]:
plot_quant_distribution(data=df, column="HEATING_COST_CURRENT")

In [ ]:
df.show()

#### Анализ столбца `HOT_WATER_COST_CURRENT`

Аналогично выполняем действия со столбцом HOT_WATER_COST_CURRENT

In [ ]:
count_nulls(data=df, column_name="HOT_WATER_COST_CURRENT")

In [174]:
df = df.filter(col("HOT_WATER_COST_CURRENT") != 0)


In [ ]:
df.show()

#### Анализ столбца `TOTAL_FLOOR_AREA`

В соответствии с описанием и содержанием датасета логично считать данный признак **количественным**. Проверим его на наличие пропущенных значений.

In [ ]:
count_nulls(data=df, column_name="TOTAL_FLOOR_AREA")

Видно, что пропуски в данном столбце отсутствуют.

Создадим функцию, позволяющую рассчитывать статистические показатели данных в столбцах и строить диаграмму "ящик с усами" для оценки наличия выбросов.

In [178]:
def plot_boxplots(data: DataFrame,
                  columns: list[str],
                  sample_fraction: float = 0.1) -> None:
    """
    Построение boxplot для нескольких столбцов в PySpark DataFrame.

    Args:
        data (DataFrame): DataFrame, содержащий данные.
        columns (list of str): Список имен столбцов для построения boxplot.
        sample_fraction (float): Доля данных для семплирования выбросов.
    
    Returns:
        None
    """
    box_data = []

    for column in columns:
        # Вычисление квантилей
        quantiles = data.approxQuantile(column, [0.25, 0.5, 0.75], 0.01)
        q1, median, q3 = quantiles

        # Вычисление IQR и границ усов
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Фильтрация выбросов
        filtered_df = data.filter((col(column) >= lower_bound) & (col(column) <= upper_bound))
        outliers_df = data.filter((col(column) < lower_bound) | (col(column) > upper_bound))

        # Вычисление минимального и максимального значений
        min_value = data.agg({column: "min"}).collect()[0][0]
        mean_value = data.agg({column: "mean"}).collect()[0][0]
        std_value = data.agg({column: "std"}).collect()[0][0]
        max_value = data.agg({column: "max"}).collect()[0][0]

        # Ограничение усов минимальным и максимальным значениями
        lower_bound = max(lower_bound, min_value)
        upper_bound = min(upper_bound, max_value)

        # Семплирование выбросов
        outliers = []
        if not outliers_df.isEmpty():
            sampled_outliers_df = outliers_df.sample(sample_fraction)
            outliers = (
                sampled_outliers_df
                .select(column)
                .limit(1000)
                .collect()
            )
            outliers = [row[column] for row in outliers]
            
            # Добавление минимального и максимального значений, если они 
            # относятся к выбросам и не присутствуют в семпле
            if min_value < lower_bound and min_value not in outliers:
                outliers.append(min_value)
            if max_value > upper_bound and max_value not in outliers:
                outliers.append(max_value)

        # Подготовка данных для axes.bxp
        box_data.append({
            'whislo': lower_bound,  # Нижняя граница усов
            'q1': q1,               # Первый квартиль
            'med': median,          # Медиана
            'q3': q3,               # Третий квартиль
            'whishi': upper_bound,  # Верхняя граница усов
            'fliers': outliers      # Выбросы
        })
        
    # Вывод статистических характеристик
    print(f"Минимальное значение:          {min_value:.2f}")
    print(f"Среднее значение:              {mean_value:.2f}")
    print(f"Среднеквадратичное отклонение: {std_value:.2f}")
    print(f"Первый квартиль:               {q1:.2f}")
    print(f"Медиана:                       {median:.2f}")
    print(f"Третий квартиль:               {q3:.2f}")
    print(f"Максимальное значение:         {max_value:.2f}")

    # Построение boxplot
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.bxp(box_data, 
           vert=False, 
           positions=range(1, len(columns) + 1), widths=0.5)
    ax.set_yticks(range(1, len(columns) + 1))
    ax.set_yticklabels(columns)
    ax.set_xlabel('Value')
    ax.set_title('Boxplots')
    ax.grid(True)
    plt.show()

In [ ]:
plot_boxplots(data=df, columns=["TOTAL_FLOOR_AREA"])

Наблюдаем небольшое количество выбросов в данных. Для более тщательного исследования создадим функцию для визуализации распределения категориального признака.

In [180]:
def plot_quant_distribution(data: DataFrame,
                            column: str,
                            num_bins: int = 200) -> None:
    """
    Построение гистограммы для количественной переменной с 
    использованием PySpark и Seaborn.

    Args:
        data (DataFrame): DataFrame с данными.
        column_name (str): Название колонки с количественной переменной.
        num_bins (int): Количество бинов для гистограммы.

    Returns:
        None
    """
    # Находим минимальное и максимальное значения колонки
    min_value = data.agg({column: "min"}).collect()[0][0]
    max_value = data.agg({column: "max"}).collect()[0][0]

    # Размер бина
    bin_size = (max_value - min_value) / num_bins

    # Добавляем колонку с номером бина
    data = data.withColumn(
        "bin", 
        floor((col(column) - min_value) / bin_size)
    )

    # Группируем по номеру бина и считаем количество строк в каждом бине
    bin_counts = data.groupBy("bin").count()

    # Преобразуем результат в Pandas DataFrame для построения гистограммы
    bin_counts_pd = bin_counts.limit(1000).toPandas()
    
    # Создаем массив границ бинов
    bin_edges = [min_value + i * bin_size for i in range(num_bins + 2)]
    
    # Преобразуем номера бинов в центры бинов
    bin_centers = [
        (bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(num_bins + 1)
    ]
    
    # Добавляем центры бинов в Pandas DataFrame
    bin_counts_pd['bin_center'] = bin_counts_pd['bin'].apply(
        lambda x: bin_centers[int(x)]
    )
    
    # Построение гистограммы с использованием Seaborn
    plt.figure(figsize=(20, 6))
    sns.histplot(data=bin_counts_pd, x="bin_center", 
                 weights="count", kde=True, bins=num_bins + 1)
    plt.xlabel("Value")
    plt.ylabel("Count")
    plt.title(f"Распределение количественного признака \"{column}\"")
    plt.grid(True)
    plt.show()

In [ ]:
plot_quant_distribution(data=df, column="TOTAL_FLOOR_AREA")

Видно, что почти все данные не превышают значения 300, но при этом наблюдается малое количество довольно сильных выбросов. Обрежем эти выбросы, установив для них максимальную границу.

In [182]:
df = df.withColumn(
    "TOTAL_FLOOR_AREA",
    when(col("TOTAL_FLOOR_AREA") > 300.0, 300.0)
        .otherwise(col("TOTAL_FLOOR_AREA"))
)

In [ ]:
plot_quant_distribution(data=df, column="TOTAL_FLOOR_AREA")

Теперь диаграмма более эффективно представляет данные в столбце.

In [ ]:
df.show()

#### Анализ столбца `NUMBER_HABITABLE_ROOMS`

Признак **категориальный**.

In [ ]:
count_nulls(data=df, column_name="NUMBER_HABITABLE_ROOMS")

Видно, что столбцы `NUMBER_HABITABLE_ROOMS` содержит пропущенные значения. Заменим их на `0.0`

In [ ]:
df = df.fillna({"NUMBER_HABITABLE_ROOMS": 0.0})
count_nulls(data=df, column_name="NUMBER_HABITABLE_ROOMS")

Создадим функцию расчета и визуализации распределения категориальных признаков.

In [ ]:
plot_cat_distribution(data=df, column_name="NUMBER_HABITABLE_ROOMS")

Видно, что недвижимость в представленном датасете имеет до 10 комнат.

123


#### Анализ столбца `NUMBER_HEATED_ROOMS`

Признак **категориальный**.

In [ ]:
count_nulls(data=df, column_name="NUMBER_HEATED_ROOMS")

Видно, что столбцы `NUMBER_HEATED_ROOMS` содержит пропущенные значения. Заменим их на `0.0`

In [ ]:
df = df.fillna({"NUMBER_HEATED_ROOMS": 0.0})
count_nulls(data=df, column_name="NUMBER_HEATED_ROOMS")

Создадим функцию расчета и визуализации распределения категориальных признаков.

In [ ]:
plot_cat_distribution(data=df, column_name="NUMBER_HEATED_ROOMS")

Видно, что недвижимость в представленном датасете имеет до 10 комнат.

#### Расчет корреляции между количественными признаками

In [ ]:
df.show()

In [196]:
def compute_and_visualize_correlation_matrix(data: DataFrame, 
                                             columns: list[str]) -> None:
    """
    Вычисляет и визуализирует корреляционную матрицу для указанных 
    колонок в DataFrame PySpark.

    Args:
        df (DataFrame): DataFrame PySpark.
        columns (list[str]): Список колонок для вычисления корреляции.

    Returns:
        None
    """
    # Вычисление корреляционной матрицы
    corr_matrix = {}
    for col1 in columns:
        corr_matrix[col1] = {}
        for col2 in columns:
            corr_value = data.select(corr(col1, col2)).collect()[0][0]
            corr_matrix[col1][col2] = corr_value

    # Преобразование корреляционной матрицы в DataFrame Pandas для визуализации
    corr_matrix_pd = pd.DataFrame(corr_matrix)

    # Построение и визуализация корреляционной матрицы
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix_pd, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Correlation Matrix')
    plt.show()

In [ ]:
compute_and_visualize_correlation_matrix(
    data=df, columns=[
        "CURRENT_ENERGY_EFFICIENCY", "HEATING_COST_CURRENT", "HOT_WATER_COST_CURRENT",  "TOTAL_FLOOR_AREA", "NUMBER_HABITABLE_ROOMS", "NUMBER_HEATED_ROOMS"
    ]
)

Корреляционная матрица демонстрирует наличие корреляции между некоторыми количественными признаками.

In [ ]:
df.show()

Посмотрим, сколько объектов осталось после преобразований датасета.

In [ ]:
df.count()

In [ ]:
df.printSchema()

Сохраняет очищенную и обработанную таблицу на диск.

In [ ]:
# Сохранение DataFrame в виде таблицы
df.writeTo("sobd_lab1_processed_table").using("iceberg").create()

In [ ]:
for table in spark.catalog.listTables():
    print(table.name)

Обратите внимание, что при необходимости созданные базу данных и таблицу можно удалить следующими командами.

In [ ]:
#spark.sql("DROP TABLE spark_catalog.Efremenkov_database.sobd_lab1_processed_table")
#spark.sql("DROP DATABASE spark_catalog.Efremenkov_database")

Останавливаем `Spark`-сессию.

In [204]:
spark.stop()